In [1]:
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from typing import List
import sys


sys.path.append("../")

from src.common_functions import *

In [2]:
# sample data
RAW_DATA_PATH = "../datasets/fraud/vehicle_fraud.csv"
DATASET_NAME = Path(RAW_DATA_PATH).stem
df = pd.read_csv(RAW_DATA_PATH)
df = df.rename(columns=dict(zip(df.columns, list(map(to_snake_case, df.columns)))))
df.head()
print(df.shape[0])
df["fraud_found_p"].value_counts()/df.shape[0]*100

15420


fraud_found_p
0    94.014267
1     5.985733
Name: count, dtype: float64

In [3]:
df.isna().any()

month                    False
week_of_month            False
day_of_week              False
make                     False
accident_area            False
day_of_week_claimed      False
month_claimed            False
week_of_month_claimed    False
sex                      False
marital_status           False
age                      False
fault                    False
policy_type              False
vehicle_category         False
vehicle_price            False
fraud_found_p            False
policy_number            False
rep_number               False
deductible               False
driver_rating            False
days_policy_accident     False
days_policy_claim        False
past_number_of_claims    False
age_of_vehicle           False
age_of_policy_holder     False
police_report_filed      False
witness_present          False
agent_type               False
number_of_suppliments    False
address_change_claim     False
number_of_cars           False
year                     False
base_pol

In [4]:
df.value_counts(["make", "fraud_found_p"])

make       fraud_found_p
Pontiac    0                3624
Toyota     0                2935
Honda      0                2622
Mazda      0                2231
Chevrolet  0                1587
Ford       0                 417
Accura     0                 413
VW         0                 275
Pontiac    1                 213
Toyota     1                 186
Honda      1                 179
Mazda      1                 123
Dodge      0                 107
Saab       0                  97
Chevrolet  1                  94
Mercury    0                  77
Accura     1                  59
Saturn     0                  52
Ford       1                  33
Nisson     0                  29
BMW        0                  14
Saab       1                  11
VW         1                   8
Jaguar     0                   6
Saturn     1                   6
Mercury    1                   6
Porche     0                   5
Mecedes    0                   3
Ferrari    0                   2
Dodge      1      

In [7]:
df.dtypes

month                    object
week_of_month             int64
day_of_week              object
make                     object
accident_area            object
day_of_week_claimed      object
month_claimed            object
week_of_month_claimed     int64
sex                      object
marital_status           object
age                       int64
fault                    object
policy_type              object
vehicle_category         object
vehicle_price            object
fraud_found_p             int64
policy_number             int64
rep_number                int64
deductible                int64
driver_rating             int64
days_policy_accident     object
days_policy_claim        object
past_number_of_claims    object
age_of_vehicle           object
age_of_policy_holder     object
police_report_filed      object
witness_present          object
agent_type               object
number_of_suppliments    object
address_change_claim     object
number_of_cars           object
year    

In [6]:
df["number_of_suppliments"]

0               none
1               none
2               none
3        more than 5
4               none
            ...     
15415           none
15416    more than 5
15417         1 to 2
15418    more than 5
15419         1 to 2
Name: number_of_suppliments, Length: 15420, dtype: object

In [6]:
y = "fraud_found_p"

# some generic cleaning
df['number_of_cars'] = df['number_of_cars'].str.split("").str[1].astype(int)
df["number_of_cars_as_int"] = df["number_of_cars"].str.split("").str[1].str.replace("m", "8").astype(int)
COLUMNS_TO_DROP = ["address_change_claim", "number_of_cars"]
assert df[y].isna().any() == False
# drop single row with Lexus as encoder can't handle cases seen in only train, while encoding test
drop_condition = ((df.make.isin(["Lexus", "Ferrari"])) | (
    df.make.isin(["BMW", "Nisson", "Mecedes"]) & (df[y] == 1)))
raw_features = df.drop(
    COLUMNS_TO_DROP, axis=1).loc[~drop_condition]

# ML tuning
# add make otherwise there are unknown brands in test set, and encoder can't handle
train, test = train_test_split(
    raw_features, test_size=0.2, random_state=0, stratify=raw_features[[y, "make", "year"]])

fit_le = LabelEncoder().fit(train[y])

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [11]:
raw_features.value_counts(["make", "fraud_found_p"])

make       fraud_found_p
Pontiac    0                3624
Toyota     0                2935
Honda      0                2622
Mazda      0                2231
Chevrolet  0                1587
Ford       0                 417
Accura     0                 413
VW         0                 275
Pontiac    1                 213
Toyota     1                 186
Honda      1                 179
Mazda      1                 123
Dodge      0                 107
Saab       0                  97
Chevrolet  1                  94
Mercury    0                  77
Accura     1                  59
Saturn     0                  52
Ford       1                  33
Nisson     0                  29
BMW        0                  14
Saab       1                  11
VW         1                   8
Mercury    1                   6
Jaguar     0                   6
Saturn     1                   6
Porche     0                   5
Mecedes    0                   3
Dodge      1                   2
Name: count, dtype

In [12]:
df.shape[0] - raw_features.shape[0]

6

In [13]:
# def build_column_transformer_for_df(train_x: pd.DataFrame, text_columns:List[str]) -> ColumnTransformer:
#     """Builds a column transformer for a pandas dataframe."""
#     # Get the categorical and numerical columns
#     categorical_columns = train_x.select_dtypes(
#         include='object').columns.to_list()
#     numerical_columns = train_x.select_dtypes(
#         include='number').columns.to_list()

#     num_prep = Pipeline(steps=[
#         ('num_imputer', SimpleImputer(strategy='median')),
#         ('scaler', StandardScaler())
#     ])

#     cat_prep = Pipeline(steps=[
#         ('cat_imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(sparse_output=False))
#     ])

#     transformer = ColumnTransformer([
#         ('text', TfidfVectorizer(), text_columns),
#         ('num', num_prep, numerical_columns),
#         ('cat', cat_prep, categorical_columns)
#     ])

#     return transformer

In [14]:
param_grid = {
    'logistic__penalty': ['l2'],
    'logistic__C': [1, 10, 100, 1000],
    'pca__n_components': list(range(6, 32, 8)),
    'under__sampling_strategy': [0.1, 0.2, 0.3, 0.5, 0.7, 1],
    'over__sampling_strategy': [0.2, 0.3, 0.4, 0.5, 0.7, 1]
}

model = LogisticRegression()

MODEL_NAME = 'logistic'

# text_columns = ["address_change_claim"]

# steps = build_column_transformer_for_df(
#     train_x=train.drop(y, axis=1))._transformers
# steps.insert(0, ('text', TfidfVectorizer(), [text_column]))
# preprocessor = ColumnTransformer(steps)

# preprocessor = build_column_transformer_for_df(train.drop(y, axis=1), text_columns)
# pipeline = build_sklearn_pipeline(
#     train, y_col_name=y, model_name=MODEL_NAME, model=model, transformer=preprocessor)
grid = sklearn_gridsearch_using_pipeline(train, y_col_name=y, model_name=MODEL_NAME,
                                  model=model, fit_le=fit_le, param_grid=param_grid, n_folds=5) #, pipeline=pipeline)
best_pipeline_log_reg = grid.best_estimator_
best_pipeline_log_reg

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


/home/maarten/.local/share/virtualenvs/End-to-end-project---Customer-churn-PLUo0CZ0/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1440 fits failed with the following error:
Traceback (most recent call last):
  File "/home/maarten/.local/share/virtualenvs/End-to-end-project---Customer-churn-PLUo0CZ0/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/maarten/.local/share/virtualenvs/End-to-end-project---Customer-churn-PLUo0CZ0/lib/python3.11/site-packages/imblearn/pipeline.py", line 293, in fit
   

Best score: 0.7921399249354913
Best parameters: {'logistic__C': 10, 'logistic__penalty': 'l2', 'over__sampling_strategy': 0.4, 'pca__n_components': 30, 'under__sampling_strategy': 0.3}


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['week_of_month',
                                                   'week_of_month_claimed',
                                                   'age', 'policy_number',
                                                   'rep_number', 'deductible',
                                                   'driver_rating', 'year']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_fre...
                                                   'days_policy_claim',
                                                   'past_number_of_claims',
                                                   'age_of_vehicle',
                                                   'age_of_policy_holder',
                                                   'police_report_filed',
                                                   'witness_present',
                                                   'agent_type',
                                                   'number_of_suppliments',
                                                   'number_of_cars',
                                                   'base_policy'])])),
                ('under', RandomUnderSampler(sampling_strategy=0.3)),
                ('over', SMOTE(sampling_strategy=0.4)),
                ('pca', PCA(n_components=30)),
                ('logistic', LogisticRegression(C=10))])

In [15]:
write_pipeline(best_pipeline_log_reg, MODEL_NAME, DATASET_NAME)

In [8]:
best_pipeline_log_reg = load_pipeline(model_name=MODEL_NAME, dataset_name=DATASET_NAME)

In [43]:
test.drop(y, axis=1).iloc[[130],:]

,month,week_of_month,day_of_week,make,accident_area,day_of_week_claimed,month_claimed,week_of_month_claimed,sex,marital_status,...,past_number_of_claims,age_of_vehicle,age_of_policy_holder,police_report_filed,witness_present,agent_type,number_of_suppliments,number_of_cars,year,base_policy
1516,Jul,2,Monday,Honda,Rural,0,0,1,Male,Single,...,none,new,16 to 17,No,No,External,none,1 vehicle,1994,All Perils


In [14]:
test.drop(y, axis=1).iloc[:130,4]

13558    Urban
9190     Rural
1485     Urban
8633     Urban
7332     Urban
         ...  
11743    Urban
1253     Urban
11856    Urban
12419    Urban
6218     Rural
Name: accident_area, Length: 130, dtype: object

In [10]:
best_pipeline_log_reg.predict(test.drop(y, axis=1).iloc[:130,:])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
best_pipeline_log_reg = load_pipeline(MODEL_NAME, DATASET_NAME)
best_pipeline_log_reg.named_steps["preprocessor"].transformers[1][1].named_steps.encoder.inverse_transform(0)

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [27]:
test.iloc[:,3].value_counts()

make
Pontiac      768
Toyota       624
Honda        561
Mazda        471
Chevrolet    336
Accura        95
Ford          90
VW            57
Saab          21
Dodge         21
Mercury       16
Saturn        11
Nisson         6
BMW            3
Jaguar         1
Mecedes        1
Porche         1
Name: count, dtype: int64

In [13]:
evaluate_model(best_pipeline_log_reg, fit_le=fit_le, test=test, y_col_name=y)

ValueError: Found unknown categories ['0'] in column 4 during transform